Download Price Paid Data and Load into DB

In [1]:
# Verifies that CSV file is up-to-date (else: D/L from Land Registry server)

from initalisation import check_vers

url = 'http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv'
dest = 'C:/Users/jamie/OneDrive/Python/Py_24/Docker/Data/'

check_vers(url,dest)

Server file is 5.1521 GB
No file exists - initiating download


100%|██████████| 5.15G/5.15G [06:27<00:00, 13.3MB/s]


In [2]:
# Copy local file into Docker volume

from initalisation import copy_file_to_docker_volume, define_name

file_name = define_name()
mount_path = "/var/lib/mysql-files"

copy_file_to_docker_volume(file_name+".csv",
                           container_name = "110e4f0dab4e",
                           local_path = "C:/Users/jamie/OneDrive/Python/Py_24/Docker/Data/",
                           mount_path = mount_path
                           )

Successfully copied C:/Users/jamie/OneDrive/Python/Py_24/Docker/Data/ to path /var/lib/mysql-files (container 110e4f0dab4e)


In [3]:
# CREATE TABLE (SQL)

from initalisation import sql_commitchanges

base = 'CREATE TABLE'+' `'+file_name.replace('_','')+'` '

fd = open('tbl_params.sql','r')
sqlFile = fd.read()
fd.close()

sql_commitchanges(base+sqlFile)

In [4]:
# LOAD DATA (SQL)

start = "LOAD DATA INFILE "+"\""+mount_path+"/"+file_name+".csv"+"\" "
mid = "INTO TABLE "+file_name.replace('_','')+" "
end = "COLUMNS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '\"' ESCAPED BY '\"' LINES TERMINATED BY '\\n'"

sql_commitchanges(start+mid+end)